# Survey memory
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code demonstrating how to add "memories" of previous questions to other simulated survey questions, to allow comparison of responses to questions where memories of prior questions are not added.

EDSL is an open-source library for simulating surveys and experiments with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## Benefits of simulated surveys
Some benefits of using AI agents to answer surveys are that agents can be presented any number of questions to answer - tirelessly! - and that questions can be presented with or without information about other responses that have already been given. This allows us to easily explore how agents respond to the same questions with different contexts.

## No-memory default
By default, EDSL survey questions are administered asynchronously to the agents and language models added to the survey when it is run. This means that questions are answered independently of each other, unless other rules have been specified. This default behavior is useful for speed and minimizing token usage (not repeating prior question contexts unnecessarily).

However, in some cases we may want an agent to have the context of some or all prior questions when answering a given question (e.g., to immitate a human's experience answering a survey). Or, we may simply want to explore the potential impact of prior question contexts by running new questions with and without memories of them.

## How it works
EDSL has several options for adding survey memories:

* `set_full_memory_mode()` adds a memory of all prior questions and answers to each new question
* `set_lagged_memory(n)` adds a memory of a specified number of prior questions and answers to each new question
* `add_targeted_memory(<new_question>, <prior_question>)` adds a memory of specific prior question to a specific new question
* `add_memory_collection(<new_question>, [<list_of_prior_questions>])` adds memories of a set of prior questions and answers to a specific new question

[Learn more about memory rules](https://docs.expectedparrot.com/en/latest/surveys.html#question-memory).

In [1]:
# pip install edsl

Installing the tools. Here we use free text and multiple choice questions; [learn more about other question types](https://docs.expectedparrot.com/en/latest/questions.html) to choose from:

In [2]:
from edsl.questions import QuestionFreeText, QuestionMultipleChoice
from edsl import Survey

Creating a survey of questions. Note that we include copies of questions in order for the results to show side-by-side responses where we have and have not added memories of prior questions:

In [3]:
q_credit = QuestionFreeText(
    question_name = "credit",
    question_text = """Have you heard about the phenomenon of hearing about an idea and 
    then believing that you are responsible for it?"""
)

q_ideas_with_credit = QuestionFreeText(
    question_name = "ideas_with_credit",
    question_text = "What's an original idea for an application for large language models."
)

q_ideas_no_credit = QuestionFreeText(
    question_name = "ideas_no_credit",
    question_text = "What's an original idea for an application for large language models."
)

q_original_with_credit = QuestionMultipleChoice(
    question_name = "original_with_credit",
    question_text = "The idea you just described - is it original?",
    question_options = ["Yes, I thought of it myself.",
                        "I may have heard about it before.",
                        "No, I have heard about it before.",
                        "I do not know."]
)

q_original_no_credit = QuestionMultipleChoice(
    question_name = "original_no_credit",
    question_text = "The idea you just described - is it original?",
    question_options = ["Yes, I thought of it myself.",
                        "I may have heard about it before.",
                        "No, I have heard about it before.",
                        "I do not know."]
)

Passing the questions to a `Survey`:

In [4]:
survey = Survey([q_credit,
                 q_ideas_with_credit,
                 q_ideas_no_credit,
                 q_original_with_credit,
                 q_original_no_credit])

## Targeted memories
Here we add memories so that the results will include (1) a response to the "original" question where the "idea" question included the context of the "credit" question and (2) a response to the "original" question where the "idea" question did *not* include the context of the "credit" question:

In [5]:
survey = (survey
          # Adding a memory of "credit" to the first iteration of "ideas" 
          .add_targeted_memory(q_ideas_with_credit, q_credit)
          # Adding memories of both "credit" and "ideas" to the first iteration of "original"
          .add_memory_collection(q_original_with_credit, [q_credit, q_ideas_with_credit])
          # Adding a memory of "ideas" where "credit" was not included to the second iteration of "original"
          .add_targeted_memory(q_original_no_credit, q_ideas_no_credit)
         )

In [6]:
results = survey.run()

In [7]:
results.select("ideas_with_credit", "original_with_credit", "ideas_no_credit", "original_no_credit").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                     ┃ answer                    ┃ answer                     ┃ answer                    ┃
┃ .ideas_with_credit         ┃ .original_with_credit     ┃ .ideas_no_credit           ┃ .original_no_credit       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ An original idea for an    │ I may have heard about it │ An original application    │ Yes, I thought of it      │
│ application for large      │ before.                   │ idea for large language    │ myself.                   │
│ language models could be a │                           │ models could be a          │                           │
│ 'Cultural Exchange         │                           │ 'Cultural Exchange         │                           │
│ Facilitator' app. This     │                           │ Facilitator' app. This app │                           │
│ platform would leverage    │                           │ would use the model's      │                           │
│ the language model's       │                           │ advanced understanding of  │                           │
│ understanding of different │                           │ language and cultural      │                           │
│ languages and cultural     │                           │ nuances to help users      │                           │
│ contexts to help users     │                           │ learn about and interact   │                           │
│ learn about and understand │                           │ with different cultures.   │                           │
│ diverse cultures. It could │                           │ It could provide real-time │                           │
│ match users with interest  │                           │ cultural insights,         │                           │
│ in specific cultural       │                           │ etiquette tips, and        │                           │
│ aspects like cuisine,      │                           │ language translation for   │                           │
│ traditions, or language,   │                           │ travelers or individuals   │                           │
│ and provide a guided       │                           │ interacting with diverse   │                           │
│ exchange program where     │                           │ communities. Additionally, │                           │
│ they can engage in         │                           │ it could include a feature │                           │
│ conversations, participate │                           │ where users can simulate   │                           │
│ in virtual cultural        │                           │ conversations with         │                           │
│ events, or collaborate on  │                           │ AI-generated characters    │                           │
│ cultural projects. The app │                           │ from different cultural    │                           │
│ would aim to promote       │                           │ backgrounds to practice    │                           │
│ cross-cultural             │                           │ language skills and        │                           │
│ understanding and empathy  │                           │ cultural awareness.        │                           │
│ by providing a space for   │                           │                            │                           │
│ meaningful dialogue and    │                           │                            │                           │
│ shared learning            │                           │                            │                           │
│ experiences.               │                           │                            │                           │
└────────────────────────────┴──────────────────────────

We can examine the question prompts to verify how the rules were applied. To see a list of all the components of the results that can be directly accessed:

In [8]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.credit',
 'answer.ideas_no_credit',
 'answer.ideas_with_credit',
 'answer.original_no_credit',
 'answer.original_with_credit',
 'comment.original_no_credit_comment',
 'comment.original_with_credit_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.credit_system_prompt',
 'prompt.credit_user_prompt',
 'prompt.ideas_no_credit_system_prompt',
 'prompt.ideas_no_credit_user_prompt',
 'prompt.ideas_with_credit_system_prompt',
 'prompt.ideas_with_credit_user_prompt',
 'prompt.original_no_credit_system_prompt',
 'prompt.original_no_credit_user_prompt',
 'prompt.original_with_credit_system_prompt',
 'prompt.original_with_credit_user_prompt',
 'question_options.credit_question_options',
 'question_options.ideas_no_credit_question_options',
 'question_options.ideas_with_credit_q

Here we inspect the system (agent) and user (question) prompts for the "credit" question:

In [9]:
results.select("credit_system_prompt", "credit_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .credit_system_prompt                                  ┃ .credit_user_prompt                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are answering questions as if you were a human. Do │ You are being asked the following question: Have you   │
│ not break character. You are an agent with the         │ heard about the phenomenon of hearing about an idea    │
│ following persona:                                     │ and                                                    │
│ {}                                                     │     then believing that you are responsible for it?    │
│                                                        │ Return a valid JSON formatted like this:               │
│                                                        │ {"answer": "<put free text answer here>"}              │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

Here we can verify that we included the context of the "credit" question in the "_with_memory" iteration of the "ideas" question:

In [10]:
results.select("ideas_with_credit_user_prompt", "original_with_credit_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .ideas_with_credit_user_prompt                         ┃ .original_with_credit_user_prompt                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are being asked the following question: What's an  │ You are being asked the following question: The idea   │
│ original idea for an application for large language    │ you just described - is it original?                   │
│ models.                                                │ The options are                                        │
│ Return a valid JSON formatted like this:               │                                                        │
│ {"answer": "<put free text answer here>"}              │ 0: Yes, I thought of it myself.                        │
│         Before the question you are now answering, you │                                                        │
│ already answered the following question(s):            │ 1: I may have heard about it before.                   │
│                 Question: Have you heard about the     │                                                        │
│ phenomenon of hearing about an idea and                │ 2: No, I have heard about it before.                   │
│     then believing that you are responsible for it?    │                                                        │
│         Answer: Yes, the phenomenon you're referring   │ 3: I do not know.                                      │
│ to is often associated with cognitive biases such as   │                                                        │
│ cryptomnesia or the illusion of originality, where a   │ Return a valid JSON formatted like this, selecting     │
│ person might unconsciously forget that an idea was     │ only the number of the option:                         │
│ encountered elsewhere and then believe it to be their  │ {"answer": <put answer code here>, "comment": "<put    │
│ own original thought. This can happen in creative      │ explanation here>"}                                    │
│ processes or when someone is exposed to a lot of       │ Only 1 option may be selected.                         │
│ information and fails to properly attribute the source │         Before the question you are now answering, you │
│ of an idea.                                            │ already answered the following question(s):            │
│                                                        │                 Question: Have you heard about the     │
│                                                        │ phenomenon of hearing about an idea and                │
│                                                        │     then believing that you are responsible for it?    │
│                                                        │         Answer: Yes, the phenomenon you're referring   │
│                                                        │ to is often associated with cognitive biases such as   │
│                                                        │ cryptomnesia or the illusion of originality, where a   │
│                                                        │ person might unconsciously forget that an idea was     │
│                                                        │ encountered elsewhere and then believe it to be their  │
│                                                        │ own original thought. This can happen in creative      │
│                                                        │ processes or when someone is exposed to a lot of       │
│                                                        │ information and fails to properly attribute the source │
│                                                       

In [11]:
results.select("ideas_no_credit_user_prompt", "original_no_credit_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .ideas_no_credit_user_prompt                           ┃ .original_no_credit_user_prompt                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are being asked the following question: What's an  │ You are being asked the following question: The idea   │
│ original idea for an application for large language    │ you just described - is it original?                   │
│ models.                                                │ The options are                                        │
│ Return a valid JSON formatted like this:               │                                                        │
│ {"answer": "<put free text answer here>"}              │ 0: Yes, I thought of it myself.                        │
│                                                        │                                                        │
│                                                        │ 1: I may have heard about it before.                   │
│                                                        │                                                        │
│                                                        │ 2: No, I have heard about it before.                   │
│                                                        │                                                        │
│                                                        │ 3: I do not know.                                      │
│                                                        │                                                        │
│                                                        │ Return a valid JSON formatted like this, selecting     │
│                                                        │ only the number of the option:                         │
│                                                        │ {"answer": <put answer code here>, "comment": "<put    │
│                                                        │ explanation here>"}                                    │
│                                                        │ Only 1 option may be selected.                         │
│                                                        │         Before the question you are now answering, you │
│                                                        │ already answered the following question(s):            │
│                                                        │                 Question: What's an original idea for  │
│                                                        │ an application for large language models.              │
│                                                        │         Answer: An original application idea for large │
│                                                        │ language models could be a 'Cultural Exchange          │
│                                                        │ Facilitator' app. This app would use the model's       │
│                                                        │ advanced understanding of language and cultural        │
│                                                        │ nuances to help users learn about and interact with    │
│                                                        │ different cultures. It could provide real-time         │
│                                                        │ cultural insights, etiquette tips, and language        │
│                                                        │ translation for travelers or individuals interacting   │
│                                                        │ with diverse communities. Additionally, it could       │
│                                                       

## Full memory
Here we create another survey to demonstrate full memory mode:

In [12]:
q1 = QuestionFreeText(
    question_name = "q1",
    question_text = "How many hours a week do you work?"
)

q2 = QuestionFreeText(
    question_name = "q2",
    question_text = "Do you have a good work-life balance?"
)

q3 = QuestionFreeText(
    question_name = "q3",
    question_text = "What changes, if any, would you like to make in your work routines?"
)

Here we show the prompts if a lagged memory rule is applied:

In [13]:
survey = Survey([q1, q2, q3])

survey_full = survey.set_full_memory_mode()

results_full = survey_full.run()

results_full.select("q1_user_prompt", "q2_user_prompt", "q3_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                              ┃ prompt                              ┃ prompt                              ┃
┃ .q1_user_prompt                     ┃ .q2_user_prompt                     ┃ .q3_user_prompt                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are being asked the following   │ You are being asked the following   │ You are being asked the following   │
│ question: How many hours a week do  │ question: Do you have a good        │ question: What changes, if any,     │
│ you work?                           │ work-life balance?                  │ would you like to make in your work │
│ Return a valid JSON formatted like  │ Return a valid JSON formatted like  │ routines?                           │
│ this:                               │ this:                               │ Return a valid JSON formatted like  │
│ {"answer": "<put free text answer   │ {"answer": "<put free text answer   │ this:                               │
│ here>"}                             │ here>"}                             │ {"answer": "<put free text answer   │
│                                     │         Before the question you are │ here>"}                             │
│                                     │ now answering, you already answered │         Before the question you are │
│                                     │ the following question(s):          │ now answering, you already answered │
│                                     │                 Question: How many  │ the following question(s):          │
│                                     │ hours a week do you work?           │                 Question: How many  │
│                                     │         Answer: As an AI, I don't   │ hours a week do you work?           │
│                                     │ work in the traditional sense or    │         Answer: As an AI, I don't   │
│                                     │ keep track of hours. I'm available  │ work in the traditional sense or    │
│                                     │ 24/7 to assist with your questions  │ keep track of hours. I'm available  │
│                                     │ and tasks.                          │ 24/7 to assist with your questions  │
│                                     │                                     │ and tasks.                          │
│                                     │                                     │                                     │
│                                     │                                     │  Prior questions and answers:       │
│                                     │                                     │ Question: Do you have a good        │
│                                     │                                     │ work-life balance?                  │
│                                     │                                     │         Answer: As an AI, I don't   │
│                                     │                                     │ have personal experiences or a      │
│                                     │                                     │ personal life, so the concept of    │
│                                     │                                     │ work-life balance doesn't apply to  │
│                                     │                                     │ me. I'm designed to be available at │
│                                     │                                     │ any time to help you with your      │
│                                     │                                     │ inquiries.                          │
└─────────────────────────────────────┴─────────────────────────────────────┴─────────────────────────────────────┘

## Lagged memory
Here we show the prompts if a lagged memory rule is used instead (note that we need to re-construct the original survey to remove the full memory rule):

In [14]:
survey = Survey([q1, q2, q3])

survey_lagged = survey.set_lagged_memory(1)

results_lagged = survey_lagged.run()

results_lagged.select("q1_user_prompt", "q2_user_prompt", "q3_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                              ┃ prompt                              ┃ prompt                              ┃
┃ .q1_user_prompt                     ┃ .q2_user_prompt                     ┃ .q3_user_prompt                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are being asked the following   │ You are being asked the following   │ You are being asked the following   │
│ question: How many hours a week do  │ question: Do you have a good        │ question: What changes, if any,     │
│ you work?                           │ work-life balance?                  │ would you like to make in your work │
│ Return a valid JSON formatted like  │ Return a valid JSON formatted like  │ routines?                           │
│ this:                               │ this:                               │ Return a valid JSON formatted like  │
│ {"answer": "<put free text answer   │ {"answer": "<put free text answer   │ this:                               │
│ here>"}                             │ here>"}                             │ {"answer": "<put free text answer   │
│                                     │         Before the question you are │ here>"}                             │
│                                     │ now answering, you already answered │         Before the question you are │
│                                     │ the following question(s):          │ now answering, you already answered │
│                                     │                 Question: How many  │ the following question(s):          │
│                                     │ hours a week do you work?           │                 Question: Do you    │
│                                     │         Answer: As an AI, I don't   │ have a good work-life balance?      │
│                                     │ work in the traditional sense or    │         Answer: As an AI, I don't   │
│                                     │ keep track of hours. I'm available  │ have personal experiences or a      │
│                                     │ 24/7 to assist with your questions  │ personal life, so the concept of    │
│                                     │ and tasks.                          │ work-life balance doesn't apply to  │
│                                     │                                     │ me. I'm designed to be available at │
│                                     │                                     │ any time to help you with your      │
│                                     │                                     │ inquiries.                          │
└─────────────────────────────────────┴─────────────────────────────────────┴─────────────────────────────────────┘

## Other survey rules
See our [documentation page about other methods](https://docs.expectedparrot.com/en/latest/surveys.html) available for `Survey` objects, including methods for applying skip/stop and other logical rules.